In [6]:
import scipy.io
import numpy as np
import datetime 
import ephem
import random
from scipy import spatial
import matplotlib.pyplot as plt
%matplotlib
from astropy.table import Table
from astropy.coordinates import EarthLocation
import astropy
from astropy.time import Time
import astropy.units as u
import numpy as np

Using matplotlib backend: MacOSX


In [22]:
def is_saa(lat, longi):  # takes input in radians
    """Return whether the satellite is in South Atlantic Anamoly or not.
    Hard bounds used from the data of RXTE and assumed that the SAA region is rectangular below the inclination of RXTE.
    
    ra - right ascention of the body in degrees
    dec - declination of the body in degrees
    time - UTC time in datetime format
    """
    # lat, longi = radec2latlong(ra, dec, time)
    #lat, longi = eci_ecef(ra, dec, time)
    lat = np.rad2deg(lat)
    longi = np.rad2deg(longi)
    if -23< lat <= -7 and -90<=longi<=-10:
        return True
    
#     if -90<=longi<=-10:
#         y = 7*longi/80 + 23/8
#         if lat < y:
#             return True
    if -100<=longi<=-35:
        y=14*longi/65+189/13
        if lat<=y:
            return True
    
    if -35<longi<-10:
        y=-14/25*longi-63/5
        if lat<y:
            return True
    
    
    if -10<=longi<=40:
        z = -9*longi/25 -43/5
        if lat<z: 
            return True
	

    if -50<= lat <=-23 and -90<=longi<=40:
        return True
    else: return False 

In [2]:
def is_visible(time,latsat,longsat, lattrig, longtrig, satephem):     #satephem is astrosat1 and astrosat2 
    satephem.compute(time)
    theta = 2*np.arcsin(6400000/(6400000+satephem.elevation))
    xsat = np.cos(longsat)*np.sin(np.pi/2 - latsat)
    ysat = np.sin(longsat)*np.sin(np.pi/2 - latsat)
    zsat = np.cos(np.pi/2 - latsat)
    satvec = (xsat,ysat,zsat)
    xtrig = np.cos(longtrig)*np.sin(np.pi/2 - lattrig)
    ytrig = np.sin(longtrig)*np.sin(np.pi/2 - lattrig)
    ztrig = np.cos(np.pi/2 - lattrig)
    trigvec = (xtrig,ytrig,ztrig)
    
    anglebetween = np.arccos(np.dot(satvec,trigvec))
    omegatrig = 2*np.pi*(1- np.cos(np.pi-anglebetween))
    omegashadow = 2*np.pi*(1- np.cos((theta/2)))
#     omegashadow = 2*np.pi*(1-np.cos(np.deg2rad(180-38.7394)))
    if omegashadow>omegatrig:
        return False
    else:
        return True

In [30]:
#is_saa(-0.5,-0.70)      #Lat = -30 , long = -40
#is_saa(-0.26,-0.70)   #Lat = -15 , long = -40
#is_saa(-0.5,-0.5)    #Lat = -30 , long = -30
#is_saa(-0.26,-1.92)   #Lat = -15 , long = -110

In [11]:
line0 = 'ASTROSAT' #Daksha
line1 = '1 40930U 15052A   22302.17850942  .00000772  00000-0  19439-4 0  9990'
line2 = '2 40930   5.9975   0.5159 0009292 339.1937  20.7998 14.76165238112787'
line3 = '2 40930   5.9975   0.5159 0009292 339.1937 200.7998 14.76165238112787'

# line0 = 'SWIFT'#BAT
# line1 = '1 28485U 04047A   22302.18405080  .00001301  00000-0  60140-4 0  9997'
# line2 = '2 28485  20.5572 277.8742 0011565 130.4507 229.6936 15.03911323708409'
# line0 = 'FGRST (GLAST)'#Fermi
# line1 = '1 33053U 08029A   22100.45970249  .00000511  00000-0  12538-4 0  9998'
# line2 = '2 33053  25.5819 140.4060 0012690 120.1881 239.9912 15.11069703542313'
astrosat1 = ephem.readtle(line0,line1,line2)
astrosat2 = ephem.readtle(line0,line1,line3)



In [35]:
is_visible('2022-01-01 00:03:13.207',0.022602884098887444,0.8306431174278259+np.pi,0.022602884098887444,0.8306431174278259,astrosat1)

False

In [7]:
#0.0226*180/np.pi

1.2948846169956605

In [8]:
#0.83064*180/np.pi

47.59216629474671